<center>
    <h3>Fundamentals of Artificial Intelligence and Knowledge Representation -
Module 3</h3>
    <h1>Flood disaster prediction</h1>
    <h5>Antonio Politano, Francesco Pieroni and Riccardo Spolaor</h5>
</center>

### Libraries import and initialization

In [1]:
import warnings
# Suppress pgmpy internal deprecated use of third party libraries.
warnings.simplefilter(action='ignore', category=FutureWarning)
# Suppress UserWarning related to machine precision calculations of percentage.
warnings.simplefilter(action='ignore', category=UserWarning)

# Import internal modules.
from utils import *
from variables import *
from extended_classes import *
from graphics import *

# Import pgmpy modules.
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination, ApproxInference
from pgmpy.sampling import GibbsSampling

# Import graphics related libraries and modules.
import matplotlib.pyplot as plt
from IPython.display import display

# Import other useful librares and modules.
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
RANDOM_STATE = 50
np.random.seed(RANDOM_STATE)

In [ ]:
%matplotlib inline

#### Node names:

In [ ]:
PER_UNIT_GDP = 'Per unit GDP'
POPULATION_DENSITY = 'Population density'
ROAD_DENSITY = 'Road density'
ELEVATION = 'Elevation'
SLOPE = 'Slope'
RAINFALL_FREQUENCY = 'Rainfall frequency'
RIVER_DENSITY = 'River density'
RAINFALL_AMOUNT = 'Rainfall amount'
FLOOD = 'Flood'

variables = [
    PER_UNIT_GDP,
    POPULATION_DENSITY,
    ROAD_DENSITY,
    ELEVATION,
    SLOPE,
    RAINFALL_FREQUENCY,
    RIVER_DENSITY,
    RAINFALL_AMOUNT,
    FLOOD
]

In [ ]:
state_names_dictionary = {
    PER_UNIT_GDP: ['High', 'Medium', 'Low'],
    POPULATION_DENSITY: ['Dense', 'Medium', 'Sparse'],
    ROAD_DENSITY: ['Dense', 'Medium', 'Sparse'],
    ELEVATION: ['High', 'Medium', 'Low'],
    SLOPE: ['Steep', 'Flat'],
    RAINFALL_FREQUENCY: ['Frequent', 'Medium', 'Rare'],
    RIVER_DENSITY: ['Dense', 'Sparse'],
    RAINFALL_AMOUNT: ['Huge', 'Medium', 'Little'],
    FLOOD: ['Yes', 'No']
}

#### Defining probability values to build all the CPDs

In [ ]:
values_dictionary = {
    PER_UNIT_GDP: [
        [0.3], 
        [0.63], 
        [0.07]
    ],
    POPULATION_DENSITY: [
        [0.7, 0.3, 0.05],
        [0.2, 0.55, 0.25],
        [0.1, 0.15, 0.7],
    ],
    ROAD_DENSITY: [
        [0.8, 0.7, 0.1, 0.6, 0.4, 0.1, 0.1, 0.05, 0.01],
        [0.19, 0.25, 0.35, 0.3, 0.5, 0.25, 0.25, 0.2, 0.15],
        [0.01, 0.05, 0.55, 0.1, 0.1, 0.65, 0.65, 0.75, 0.84]
    ],
    ELEVATION: [
        [0.15],
        [0.1], 
        [0.75]
    ],
    SLOPE: [
        [0.75, 0.6, 0.05], 
        [0.25, 0.4, 0.95]
    ],
    RAINFALL_FREQUENCY: [
        [0.3], 
        [0.6], 
        [0.1]
    ],
    RIVER_DENSITY: [
        [0.4], 
        [0.6]
    ],
    RAINFALL_AMOUNT: [
        [0.7, 0.5, 0.55, 0.3, 0.1, 0.01],
        [0.2, 0.25, 0.3, 0.4, 0.3, 0.04],
        [0.1, 0.25, 0.15, 0.3, 0.6, 0.95]
    ],
    FLOOD: [
        [0.07, 0.03, 0.005, 0.2, 0.1, 0.008, 0.05, 0.009, 0.0005, 0.13, 0.08, 0.006, 0.008, 0.002, 0.0001, 0.1, 0.04, 0.0002],
        [0.93, 0.97, 0.995, 0.8, 0.9, 0.992, 0.95, 0.991, 0.9995, 0.87, 0.92, 0.994, 0.992, 0.998, 0.9999, 0.9, 0.96, 0.9998]
    ],
}

In [ ]:
edges = [
    (PER_UNIT_GDP, POPULATION_DENSITY),
    (PER_UNIT_GDP, ROAD_DENSITY), 
    (POPULATION_DENSITY, ROAD_DENSITY),
    (ROAD_DENSITY, FLOOD),
    (ELEVATION, SLOPE),
    (SLOPE, FLOOD),
    (RAINFALL_FREQUENCY, RAINFALL_AMOUNT),
    (RIVER_DENSITY, RAINFALL_AMOUNT),
    (RAINFALL_AMOUNT, FLOOD)
]

In [ ]:
evidence_dictionary = {
    PER_UNIT_GDP: None,
    POPULATION_DENSITY: [PER_UNIT_GDP],
    ROAD_DENSITY: [PER_UNIT_GDP, POPULATION_DENSITY],
    ELEVATION: None,
    SLOPE: [ELEVATION],
    RAINFALL_FREQUENCY: None,
    RIVER_DENSITY: None,
    RAINFALL_AMOUNT: [RAINFALL_FREQUENCY, RIVER_DENSITY],
    FLOOD: [ROAD_DENSITY, SLOPE, RAINFALL_AMOUNT]
}

In [ ]:
cpds = {v: get_tabular_cpd(v, state_names_dictionary, values_dictionary, evidence_dictionary) for v in variables}

In [ ]:
for k, v in cpds.items():
    print('CPD Table for variable: {}'.format(k))
    display(cpd_to_pandas(v))
    print()

## Defining the model structure
We can define the network by just passing a list of edges and then add the already defined CPDs

In [ ]:
model = ExtendedBayesianNetwork(edges)
model.add_cpds(*[cpds[k] for k in cpds])

In [ ]:
print('The model has been correctly developed: {}.'.format(model.check_model()))

The model consists of the following Bayesian Network:

In [ ]:
display_bayesian_network(model)

## Indepencies
Getting all the independencies given the parent nodes in the network.

In [ ]:
print('Independecies given the parent nodes shown using the function local_independencies():')
model.local_independencies(variables)

### Active Trails
To see the flow of information, the active trail of each node is shown:

In [ ]:
for v in variables:
    print(model.active_trail_nodes(v))

### V-Structure activation
V - structure activated from the evidence "Rainfall amount", therefore river density influences rainfall frequency 

In [ ]:
active_trail_nodes = model.active_trail_nodes(RIVER_DENSITY, observed=RAINFALL_AMOUNT)
print(f'Trail of influence for variable {RIVER_DENSITY}: {active_trail_nodes}')
display_v_structure(model, RIVER_DENSITY, RAINFALL_AMOUNT, active_trail_nodes)

Example of blocked flow of information, since Population density is shielded by "Road density" and "Per unit GDP"

In [ ]:
print(model.active_trail_nodes(POPULATION_DENSITY, observed=[ROAD_DENSITY, PER_UNIT_GDP]))

This result is obtained since the given evidence is exactly the Markov Blanket of "Population density", which is confermed by the the pgmpy method get_markov_blanket.

In [ ]:
model.get_markov_blanket(POPULATION_DENSITY)

### Showing Markov Blankets

In [ ]:
def display_markov_blanket (variable, markov_blanket):
    color_map = []
    for node in model.nodes:
        if node == variable:
            color_map.append('green')
        elif node in markov_blanket: 
            color_map.append('red') 
        else:
            color_map.append('blue')
            
    nx.draw(model, node_size = 2500, node_color=color_map, with_labels = True)

    plt.show()

In [ ]:
for v in variables:
    markov_blanket = model.get_markov_blanket(v)
    display_markov_blanket (model, v, markov_blanket)
    print()

# Exact inference

In [ ]:
exact_infer = VariableElimination(model)

**P(Flood)**

In [ ]:
print_exact_inference(FLOOD, exact_infer)

**P(Flood | Road density = Dense)**

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={ROAD_DENSITY: 'Dense'})

**P(Flood | River density = Dense)**

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={RIVER_DENSITY: 'Dense'})

**P(Flood | Road density = Dense, Slope = Flat)**

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Flat'})

**P(Flood | Road density = Dense, Slope = Flat, Rainfall Frequency = Frequent)**

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Flat', RAINFALL_FREQUENCY: 'Frequent'})


**P(Flood | Road density = Medium, Slope = Flat, Rainfall Frequency = Frequent)**

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={ROAD_DENSITY: 'Medium', SLOPE: 'Flat', RAINFALL_FREQUENCY: 'Frequent'})

**P(Flood | Road density = Dense, Slope = Steep, Rainfall Frequency = Frequent)**

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'})

**P(Slope)**

In [ ]:
print_exact_inference(SLOPE, exact_infer)

**P(Slope | Flood = Yes)**

In [ ]:
print_exact_inference(SLOPE, exact_infer, evidence={FLOOD: 'Yes'})

**P(Rainfall Amount | Flood = Yes)**

In [ ]:
print_exact_inference(RAINFALL_AMOUNT, exact_infer, evidence={FLOOD: 'Yes'})

**P(Rainfall Frequency | Rainfall Amount = Huge)**

In [ ]:
print_exact_inference(RAINFALL_FREQUENCY, exact_infer, evidence={RAINFALL_AMOUNT: 'Huge'})

**P(Rainfall Frequency | Rainfall Amount = Little)**

In [ ]:
print_exact_inference(RAINFALL_FREQUENCY, exact_infer, evidence={RAINFALL_AMOUNT: 'Little'})

# Approximate Inference

This section evaluates the approximate inference on a series of variables given evidence. The calculation is performed by using the Approximate Inference method `ApproxInference.query` which applies the *Negation Sampling*. The results are compared with their *Exact Inference* counterpart.

### Using Negation Sampling

In [ ]:
approx_infer_sampling = ExtendedApproxInference(model)

#### P(Flood)

In [ ]:
print_exact_inference(FLOOD, exact_infer)
print()
print_approximate_inference(FLOOD, approx_infer_sampling, n_samples=1_000, random_state=RANDOM_STATE)

The result shows that approximate inference gives a very good estimation of the probability distribution of Flood, since the values are very close the ones we get with exact inference. The main reason is that we used a very high number of samples, which approximates well the limit to infinity

#### P(Flood | Road Density = Dense, Slope = Steep, Rainfall Frequency = Frequent)

In [ ]:
print_exact_inference(FLOOD, exact_infer, evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'})

print_approximate_inference(FLOOD, 
                            approx_infer_sampling, 
                            evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, 
                            n_samples=1_000, 
                            random_state=RANDOM_STATE)

As for the case above, the estimation using approximate inference with negation sampling is very good, since we used many samples. Unfortunatelly, this results in having a longer execution time than using exact inference, so there is no benefit in using approximate inference with those many samples.
Rather, we can dramatically decrease the number of samples and see what happens

In [ ]:
print_approximate_inference(FLOOD, approx_infer_sampling, 
                            evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, 
                            n_samples=50, 
                            random_state=RANDOM_STATE)

Approximate inference with negation sampling does not work quiet well, giving results worse than the second case where more samples were given. In addition, the execution time has not changed a lot.

### Second case
#### P(Flood | Per unit GDP = Low, Elevation = Medium, Rainfall Frequency = Rare)

In [ ]:
"""start_time = time.time_ns()

print("Exact Inference to find P(Flood | Per unit GDP = Dense, Slope = Flat, Rainfall Frequency = Frequent)\n")
print(exact_infer.query([FLOOD], 
                        evidence={PER_UNIT_GDP: 'Low', ELEVATION: 'Medium', RAINFALL_FREQUENCY: 'Rare'}, 
                        show_progress=False
))

print(f"--- {(time.time_ns() - start_time) / 1_000_000_000} seconds ---")

start_time = time.time_ns()

print("Approximate Inference with sampling to find P(Flood)\n")
print(approx_infer_sampling.query(variables=[FLOOD], 
                                  evidence={PER_UNIT_GDP: 'Low', ELEVATION: 'Medium', RAINFALL_FREQUENCY: 'Rare'}, 
                                  n_samples=1_000, show_progress=False, seed=RANDOM_STATE
))


print(f"--- {(time.time_ns() - start_time)  / 1_000_000_000} seconds ---")""";

As for the case above, the estimation using approximate inference with negation sampling is very good, since we used many samples. Unfortunatelly, this results in having a longer execution time than using exact inference, so there is no benefit in using approximate inference with those many samples.
Rather, we can dramatically decrease the number of samples and see what happens

### Third case (few samples)
#### P(Flood | Per unit GDP = Low, Elevation = Medium, Rainfall Frequency = Rare)

In [ ]:
"""start_time = time.time_ns()

print("Exact Inference to find P(Flood | Per unit GDP = Dense, Slope = Flat, Rainfall Frequency = Frequent)\n")
print(exact_infer.query([FLOOD], 
                        evidence={PER_UNIT_GDP: 'Low', ELEVATION: 'Medium', RAINFALL_FREQUENCY: 'Rare'}, 
                        show_progress=False
))

print(f"--- {(time.time_ns() - start_time) / 1_000_000_000} seconds ---")

start_time = time.time_ns()

print("Approximate Inference with sampling to find P(Flood)\n")
print(approx_infer_sampling.query(variables=[FLOOD], 
                                  evidence={PER_UNIT_GDP: 'Low', ELEVATION: 'Medium', RAINFALL_FREQUENCY: 'Rare'}, 
                                  n_samples=50, show_progress=False, seed=RANDOM_STATE
))


print(f"--- {(time.time_ns() - start_time)  / 1_000_000_000} seconds ---")""";

Approximate inference with negation sampling does not work quiet well, giving results worse than the second case where more samples were given. In addition, the execution time has not changed a lot.

#### P(Flood | Road density = Dense, Slope = Flat, Rainfall Frequency = Frequent)
* Let's discuss the fact that the approximate inference should be maybe quicker rather than exact inference

In [ ]:
"""start_time = time.time_ns()

print("Exact Inference to find P(Flood|Road density = Dense, Slope = Flat, Rainfall Frequency = Frequent)\n")
print(exact_infer.query([FLOOD], 
                        evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, 
                        show_progress=False
))

print(f"--- {(time.time_ns() - start_time) / 1_000_000_000} seconds ---")

start_time = time.time_ns()

print("Approximate Inference with sampling to find P(Flood)\n")
print(approx_infer_sampling.query(variables=[FLOOD], 
                                  evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, 
                                  n_samples=50, show_progress=False, seed=RANDOM_STATE
))


print(f"--- {(time.time_ns() - start_time)  / 1_000_000_000} seconds ---")""";

As for the case above, the estimation using approximate inference is very good, since we used many samples. Unfortunatelly, this results in having a longer execution time than using exact inference, so there is no benefit in using approximate inference with those many samples.
Rather, we can dramatically decrease the number of samples and see what happens

In [ ]:
"""start_time = time.time_ns()

print("Approximate Inference with sampling to find P(Flood)\n")
print(approx_infer_sampling.query(variables=[FLOOD], evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, n_samples=1000, show_progress=False, seed=RANDOM_STATE))

print(f"--- {(time.time_ns() - start_time)  / 1_000_000_000} seconds ---")""";

Approximate inference still works quiet well, but gives results worse than the first try with more samples. In addition, the execution time is perfectly comparable to the one of exact inference

## Using Likelihood Weighting

This section evaluates the approximate inference on a series of variables given evidence using the Likelihood Weighting method. The calculation is performed by using the `ApproxInferenceWeightedSampling.query` which extends the `ApproxInference.query` method by giving the option to use the *Likelihood Weighting* method on the *Bayesian Model*. The results are compared with their *Exact Inference* counterpart.

In [ ]:
approx_infer_sampling = ExtendedApproxInference(model)

#### P(Flood)

In [ ]:
print_exact_inference(FLOOD, exact_infer)
print()
print_approximate_inference(FLOOD, 
                            approx_infer_sampling, 
                            n_samples=1_000, 
                            random_state=RANDOM_STATE, 
                            use_weighted_likelihood=True)

The result shows that approximate inference gives a very good estimation of the probability distribution of Flood, since the values are very close the ones we get with exact inference. The main reason is that we used a very high number of samples, which approximates well the limit to infinity

#### P(Flood | Road Density = Dense, Slope = Steep, Rainfall Frequency = Frequent)

In [ ]:
print_exact_inference(FLOOD, 
                      exact_infer, 
                      evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'})

print()

print_approximate_inference(FLOOD, 
                            approx_infer_sampling,
                            evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, 
                            n_samples=1_000, 
                            random_state=RANDOM_STATE, 
                            use_weighted_likelihood=True)

As for the case above, the estimation using approximate inference with negation sampling is very good, since we used many samples. Unfortunatelly, this results in having a longer execution time than using exact inference, so there is no benefit in using approximate inference with those many samples.
Rather, we can dramatically decrease the number of samples and see what happens

In [ ]:
print_approximate_inference(FLOOD, 
                            approx_infer_sampling, 
                            evidence={ROAD_DENSITY: 'Dense', SLOPE: 'Steep', RAINFALL_FREQUENCY: 'Frequent'}, 
                            n_samples=50, 
                            random_state=RANDOM_STATE, 
                            use_weighted_likelihood=True)

Approximate inference with weighted likelihood sampling does not work quiet well, giving results worse than the second case where more samples were given. In addition, the execution time has not changed a lot.

#### P(Flood | Per unit GDP = Low, Elevation = Medium, Rainfall Frequency = Rare)
* Let's discuss the fact that the approximate inference should be maybe quicker rather than exact inference

In [ ]:
"""start_time = time.time_ns()

evidence_to_use = {PER_UNIT_GDP: 'Low', ELEVATION: 'Medium', RAINFALL_FREQUENCY: 'Rare'}

print("Exact Inference to find P(Flood|Road density = Dense, Slope = Flat, Rainfall Frequency = Frequent)\n")
print(exact_infer.query([FLOOD], evidence=evidence_to_use, show_progress=False))

print(f"--- {(time.time_ns() - start_time) / 1_000_000_000} seconds ---")

start_time = time.time_ns()

print("Approximate Inference with sampling to find P(Flood)\n")
print(approx_infer_sampling.query(variables=[FLOOD], evidence=evidence_to_use, n_samples=100, show_progress=False, 
                                  use_weighted_sampling=True, seed=RANDOM_STATE))


print(f"--- {(time.time_ns() - start_time)  / 1_000_000_000} seconds ---")""";

## Using Markov Chain Monte Carlo Method

In [ ]:
gibbs = GibbsSampling(model)
gibbs_df = gibbs.sample(size=10_000, seed=RANDOM_STATE)

In [ ]:
gibbs_df['Flood'].value_counts()

#### P(Flood)

In [ ]:
series_Flood = gibbs_df['Flood'].value_counts()

#Flood  Yes:0, No:1
print('Phi(Flood = Yes): ',series_Flood[0]/(sum(series_Flood)) )
print('Phi(Flood = No): ',series_Flood[1]/(sum(series_Flood)) )

#### P(Flood  | Per unit GDP = High)

In [ ]:
#GDP High: 0, Medium: 1, Low: 2
series_Flood_GDP = gibbs_df.loc[gibbs_df['Per unit GDP'] == 0]['Flood'].value_counts()

#Flood  Yes:0, No:1
print('Phi(Flood = Yes | Per unit GDP = High): ',series_Flood_GDP[0]/(sum(series_Flood_GDP)) )
print('Phi(Flood = No | Per unit GDP = High): ',series_Flood_GDP[1]/(sum(series_Flood_GDP)) )

# Use Case

In this section an use case for the implemented Bayesian Network is presented. 

We decided to test the performance of our qualitative built network by using relevant information related to each municipality of the Italian Veneto region.
In particular, the chance of flooding on a yearly basis of each municipality is computed through *Exact inference* given as evidence frequent rainfalls for the period (**Rainfall Frequency = Frequent**) and each municipality information about the **Per unit GDP**, **Population density**, **Slope** and **River density**.

$$ \forall \ m \in \text{Municipalities}, P(\text{Flood}_m\ | \ \text{Per unit GDP}_m, \text{Population density}_m, \text{Slope}_m,\text{River density}_m, \text{Rainfall frequency = Frequent})$$

## Municipalities data
The municipalities data needed to perform the inference is obtained throgh different reliabe sources (e.g.: the *Ministry of Economy and Finance* of Italy, the *Italian National Institute of Statistics* or the Veneto Region official web platform) and it is collected in a `pandas` `DataFrame`.

The sources of each piece of information are expressed below:
* Municipalities general information and regional incomes: https://www1.finanze.gov.it/finanze/analisi_stat/public/index.php?tree=2020
* Municipalities surfaces: https://data.europa.eu/data/datasets/superficie_territoriale_in_kmq_dei_comuni_del_veneto?locale=it
* Municipalities population: https://www.istat.it/it/archivio/243448
* Municipalities water surfaces: https://www.regione.veneto.it/web/ambiente-e-territorio/scheda-dati
* Municipalities altitudes: https://www.istat.it/it/archivio/156224

Missing data has been filled according to the municipalities information given by the following websites:
* https://it.wikipedia.org/wiki/Pagina_principale
* https://www.cittaeborghi.it/it/

Useful constants for data manipulation are expressed below

In [ ]:
ISTAT_VENETO_CODE = 5
MUNICIPALITY_NAME_COLUMN = 'Municipality Name'
ISTAT_CODE_COLUMN = 'Istat code'
PROVINCE_COLUMN = 'Province'
SURFACE_COLUMN = 'Surface (km2)'

### Main dataframe
The main dataframe (`df`) is created by exploiting the information given by the region's income data (`income_df`)

The `income_df` is created and inspected.

In [ ]:
income_df = pd.read_csv('./data/Redditi_e_principali_variabili_IRPEF_su_base_comunale_CSV_2019.csv', sep=';', index_col=False)
income_df.head()

`income_df` is reduced to the sole municipalities of the Veneto region, which are 563.

In [ ]:
income_df = income_df[income_df['Codice Istat Regione'] == ISTAT_VENETO_CODE]
income_df.reset_index(inplace=True)
income_df.shape

The main dataframe (`df`) is created using the `income_df`.

In [ ]:
df = income_df[['Denominazione Comune', 'Codice Istat Comune', 'Sigla Provincia']]

df = df.rename(columns={
    'Denominazione Comune': MUNICIPALITY_NAME_COLUMN, 
    'Codice Istat Comune': ISTAT_CODE_COLUMN, 
    'Sigla Provincia': PROVINCE_COLUMN
})

df.head()

No missing values are present in the newly created `df`.

In [ ]:
print('Number of NaN values in df: \n{}'.format(df.isna().sum()))

### Per unit GDP information

Information needed to computed the **Per unit GDP** values is inspected, and it is available for each municipality

In [ ]:
print('Number of NaN values regarding the total amount of income: {}'.format(
    income_df['Reddito imponibile - Ammontare in euro'].isna().sum())
     )
print('Number of NaN values regarding the total amount of tax payers: {}'.format(income_df['Numero contribuenti'].isna().sum()))

The **Per unit GDP** data is computed by dividing for each region the total income by the number of tax payers given by the `income_df` and the column is added to the main `df`.

In [ ]:
df[PER_UNIT_GDP] = income_df['Reddito imponibile - Ammontare in euro'] / income_df['Numero contribuenti']
df.head()

### Population density information

Information needed to computed the **Population density** values is obtained by firstly creating the `population_density_df` and then merging the density per $km^2$ information into the main `df`.

In [ ]:
population_density_df = pd.read_excel('./data/05_Veneto_Allegato-statistico.xlsx', 'Appendice 2',  skiprows = 10)
population_density_df = population_density_df[['ProCom', 'Densità']]
population_density_df.head()

In [ ]:
df = pd.merge(df, population_density_df, left_on=ISTAT_CODE_COLUMN, right_on='ProCom', how='left').drop('ProCom', axis=1)

df = df.rename(columns={'Densità': POPULATION_DENSITY})

Missing information is filled and the main `df` is shown.

In [ ]:
print('Rows of df where "{}" is NaN.'.format(POPULATION_DENSITY))

df[df[POPULATION_DENSITY].isna()]

In [ ]:
df.loc[558, POPULATION_DENSITY] = 80.15
df.loc[559, POPULATION_DENSITY] = 175.74
df.loc[560, POPULATION_DENSITY] = 53.28
df.loc[561, POPULATION_DENSITY] = 307.63
df.loc[562, POPULATION_DENSITY] = 76.32

In [ ]:
df.head()

### Slope information

Information needed to computed the **Slope** values is obtained by firstly creating the `elevation_df`, next and then merging the altitude range information of each municipality (expressed in $mts$) into the main `df`.

In [ ]:
elevation_df = pd.read_excel('./data/Elab_Altimetrie_DEM.xlsx')
elevation_df.head()

In [ ]:
elevation_df = elevation_df[elevation_df['COD_REG'] == ISTAT_VENETO_CODE]
elevation_df = elevation_df.rename(columns={'RANGE': SLOPE,})
elevation_df = elevation_df[['PRO_COM', SLOPE]]

df = pd.merge(
    df, 
    elevation_df, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='PRO_COM', 
    how = 'left'
).drop('PRO_COM', axis=1)

Missing information is filled and the main `df` is shown.

In [ ]:
print('Rows of df where "{}" is NaN.'.format(SLOPE))

df[df[SLOPE].isna()]

In [ ]:
df.loc[551, SLOPE] = 1625 - 175
df.loc[552, SLOPE] = 2502 - 400 
df.loc[553, SLOPE] = 3210 - 593
df.loc[554, SLOPE] = 2471 - 380
df.loc[555, SLOPE] = 320 - 15
df.loc[556, SLOPE] = 445 - 15
df.loc[557, SLOPE] = 22 - 7

# Ulterior missing (and unavailable) information was filled by the mean value of "Slope" across the whole dataset.
df[SLOPE].fillna(df[SLOPE].mean(), inplace=True)

In [ ]:
df.head()

### River density

Information needed to computed the **River density** values is obtained by firstly creating the `river_density_df`.

In [ ]:
river_density_df = pd.read_excel('./data/codiceISTAT_schedaLR14_2017.ods', 'ISTAT')
river_density_df.head()

Next, the **River density** information is obtained by dividing the surface of water of each municipality (`classe_5`) by its surface (`Superficie Territoriale`) and the `river_density_df` dataset is reduced with the solely needed features.

In [ ]:
river_density_df[RIVER_DENSITY] = river_density_df['classe_5'] / river_density_df['Superficie Territoriale']
river_density_df = river_density_df[['cod. comune', RIVER_DENSITY]]
river_density_df.head()

The **River density** information of each municipality (expressed in a value between 0 and 1) is merged into the main `df`.

In [ ]:
df = pd.merge(
    df, 
    river_density_df, 
    left_on=ISTAT_CODE_COLUMN,
    right_on='cod. comune', 
    how = 'left'
).drop('cod. comune', axis=1)

Missing information is filled by the mean value of **River density** across the dataset and the main `df` is shown.

In [ ]:
print('Rows of df where "{}" is NaN.'.format(RIVER_DENSITY))

df[df[RIVER_DENSITY].isna()]

In [ ]:
df[RIVER_DENSITY].fillna(df[RIVER_DENSITY].mean(), inplace=True)

df.head()

## Data discretization
The municipalities data collected in `df` is discretized according to the information used to build the *Bayesian Network* or their *exact inference* when the variables have parent nodes in the network.

In [ ]:
per_unit_gdp_inference = exact_infer.query([PER_UNIT_GDP], show_progress=False)
population_density_inference = exact_infer.query([POPULATION_DENSITY], show_progress=False)
slope_inference = exact_infer.query([SLOPE], show_progress=False)
river_density_inference = exact_infer.query([RIVER_DENSITY], show_progress=False)

The infered data shown above is used to discretize the `df` data according to specific quantiles.

In [ ]:
apply_discrete_values(PER_UNIT_GDP, df, [
    df[PER_UNIT_GDP].min(),
    df[PER_UNIT_GDP].quantile(per_unit_gdp_inference.values[-1]),
    df[PER_UNIT_GDP].quantile(1 - per_unit_gdp_inference.values[0]),
    df[PER_UNIT_GDP].max()
], state_names_dictionary)

apply_discrete_values(SLOPE, df, [
    df[SLOPE].min(),
    df[SLOPE].quantile(slope_inference.values[-1]),
    df[SLOPE].max()
], state_names_dictionary)

apply_discrete_values(POPULATION_DENSITY, df, [
    df[POPULATION_DENSITY].min(),
    df[POPULATION_DENSITY].quantile(population_density_inference.values[-1]),
    df[POPULATION_DENSITY].quantile(1 - population_density_inference.values[0]),
    df[POPULATION_DENSITY].max()
], state_names_dictionary)

apply_discrete_values(RIVER_DENSITY, df, [
    df[RIVER_DENSITY].min(),
    df[RIVER_DENSITY].quantile(river_density_inference.values[-1]),
    df[RIVER_DENSITY].max()
], state_names_dictionary)

In [ ]:
df.head()

## Likelihood of flooding

The chance of flooding on a yearly basis of each municipality is computed through *Exact inference* given as evidence frequent rainfalls for the period (**Rainfall Frequency = Frequent**) and each municipality information about the **Per unit GDP**, **Population density**, **Slope** and **River density**.

$$ \forall \ m \in \text{Municipalities}, P(\text{Flood}_m\ | \ \text{Per unit GDP}_m, \text{Population density}_m, \text{Slope}_m,\text{River density}_m, \text{Rainfall frequency = Frequent})$$

In [ ]:
df['flood_likelihood'] = df.apply(
    lambda row: exact_infer.query([FLOOD], evidence={
        PER_UNIT_GDP: row[PER_UNIT_GDP], 
        POPULATION_DENSITY: row[POPULATION_DENSITY], 
        SLOPE: row[SLOPE], 
        RIVER_DENSITY: row[RIVER_DENSITY],
        RAINFALL_FREQUENCY: 'Frequent'
    }, show_progress=False).get_value(Flood='Yes'), 
    axis=1
)

In [ ]:
df.head()

The `geopandas_df` is created in order to represent each municipality as a geometrical 2-dimensional shape and to plot a map of the Veneto region divided in municipalities with information about the flood chances.

The data has been fetched from the following site:
* https://github.com/openpolis/geojson-italy

In [ ]:
geopandas_df = gpd.read_file('https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_R_5_municipalities.geojson')

In [ ]:
geopandas_df.head()

The number of municipalities of `geopandas_df` corresponds to the ones of `df`.

In [ ]:
print((geopandas_df.shape[0], df.shape[0]))

The **Flood likelihood** information for each municipality (expressed in a value between 0 and 1) is merged into the `geopandas_df`.

In [ ]:
geopandas_df = pd.merge(
    geopandas_df, 
    df[['Istat code','flood_likelihood']], 
    left_on='com_istat_code_num',
    right_on='Istat code', 
    how = 'left'
).drop('Istat code', axis=1)

In [ ]:
geopandas_df.head()

### Flood chances heatmap

Finally, a heatmap showing the probability of flood for each municipality is shown.

The results are significative, since it is evdent how the northern areas where the slope of the surface is steeper have lower chances of being flooded, although there may be some exception, like in places near *Belluno*, which are zones with a high elevation, but with a quite flat surface resulting in higher probabilities of flooding.

More densely populated and richer areas such as some chief towns (*Venezia*, *Verona*, *Vicenza*, *Padova* and *Rovigo*) have the highest likelihood of experiencing floods, whereas poorer and less populated although quietly flat areas in the southern regions show medium probabilities of being flooded.

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=-1)

ax = geopandas_df.plot(column='flood_likelihood', cmap='seismic', ax=ax, legend= True, cax=cax)

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

provinces = ['Venezia', 'Verona', 'Vicenza', 'Padova', 'Treviso', 'Rovigo', 'Belluno']

for idx, row in geopandas_df[geopandas_df['name'].isin(provinces)].iterrows():
    coords = row['geometry'].representative_point().coords[:]
    coords = coords[0]
    ax.annotate(row['prov_acr'], xy=coords, horizontalalignment='center')
    
fig.suptitle('Flood probability in Veneto region by municipality', y=0.86, x=0.56)
plt.show()